In [8]:
import os

#data processing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# plotting
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

#training
import tensorflow as tf

#Image processing
from PIL import Image
import cv2



In [9]:
#import data from kaggle

import kagglehub

# Download latest version
path = kagglehub.dataset_download("tongpython/cat-and-dog")

print("Path to dataset files:", path)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 218M/218M [00:20<00:00, 11.2MB/s]

Extracting files...


Path to dataset files: /Users/ansinha/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1


In [14]:
# counting files
path_to_cat_folder = path+"/training_set/training_set/cats"
path_to_dog_folder =path+"/training_set/training_set/dogs"

path_to_cat_folder, dirs, cat_files = next(os.walk(path_to_cat_folder))
file_count_cat = len(cat_files)
print( "Number of cat files:", file_count_cat)

path_to_dog_folder, dirs, dog_files = next(os.walk(path_to_dog_folder))
file_count_dog = len(dog_files)
print("Number of dog files:", file_count_dog)

Number of cat files: 4001
Number of dog files: 4006


In [20]:
#Processing cat images
os.makedirs(path+'/working/cats/', exist_ok=True)
os.makedirs(path+'/working/dogs/', exist_ok=True)
print(f"Successfully created directories")


for filename in cat_files:
  if filename == '_DS_Store':
    print("Skipped:", filename)
    continue
      


    
  image_cat = Image.open(path_to_cat_folder + "/" + filename)
  resized_cat_image = image_cat.resize((224, 224))

  resized_cat_image.save(path+'/working/cats/' + filename)

print("Cat Files resizing and grayscaling Done")

Successfully created directories
Skipped: _DS_Store
Cat Files resizing and grayscaling Done


In [21]:
#Processing dog images
for filename in dog_files:
  if filename == '_DS_Store':
    print("Skipped:", filename)
    continue

  image_dog = Image.open(path_to_dog_folder + "/" + filename)
  resized_dog_image = image_dog.resize((224, 224))
  resized_dog_image.save(path+'/working/dogs/' + filename)

print("Dog Files resizing and grayscaling Done")

Skipped: _DS_Store
Dog Files resizing and grayscaling Done


In [23]:
#creating X, y, X_test, y_test

cat_training_filenames = os.listdir(path+"/working/cats")
dog_training_filenames = os.listdir(path+"/working/dogs")

X = []
Y = []

for filename in cat_training_filenames:
  if filename == '_DS_Store':
    print("Skipped:", filename)
    continue
  X.append(np.array(Image.open(path+"/working/cats/" + filename)))
  Y.append(0)

for filename in dog_training_filenames:
  if filename == '_DS_Store':
    print("Skipped:", filename)
    continue
  X.append(np.array(Image.open(path+"/working/dogs/" + filename)))
  Y.append(1)

print("X and Y created")


X and Y created


#Train Test Split

In [24]:
random_indices = np.random.permutation(len(X))
X = np.array(X)[random_indices]
Y = np.array(Y)[random_indices]

X=X/255 # rescaling

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [25]:
print(X.shape)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(8005, 224, 224, 3)
(6404, 224, 224, 3)
(6404,)
(1601, 224, 224, 3)
(1601,)


#Building the Neural Network

In [27]:
import tensorflow as tf
import tensorflow_hub as hub

In [28]:
# mobilenet_V2_model = "https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/035-128-classification/2"
# pretrained_model = hub.KerasLayer(mobilenet_V2_model, input_shape=(224, 224, 3), trainable=False)
num_of_classes = 2
base = tf.keras.applications.MobileNetV2(
    include_top=False,
    pooling='avg',                # global average pool output
    input_shape=(224, 224, 3),
    weights='imagenet'
)
base.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = base(inputs, training=False)

outputs = tf.keras.layers.Dense(2, activation='softmax')(x) # alternative
model = tf.keras.Model(inputs, outputs)
model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         2,562 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,260,546 (8.62 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [29]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [30]:
model.fit(X_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 34s 156ms/step - accuracy: 0.9428 - loss: 0.1489
Epoch 2/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 35s 173ms/step - accuracy: 0.9832 - loss: 0.0531
Epoch 3/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 32s 161ms/step - accuracy: 0.9858 - loss: 0.0430
Epoch 4/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 34s 170ms/step - accuracy: 0.9897 - loss: 0.0308
Epoch 5/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 35s 173ms/step - accuracy: 0.9913 - loss: 0.0312


In [31]:
model.evaluate(X_test, y_test)

51/51 ━━━━━━━━━━━━━━━━━━━━ 9s 168ms/step - accuracy: 0.9803 - loss: 0.0498


[0.048777107149362564, 0.9806370735168457]

#Building a Predictive System

In [45]:
# Single cell: FileUpload widget -> preprocess -> predict -> print Cat/Dog
import io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# -------------------------
# REQUIREMENT: Load your model before using the widget.
# Example (uncomment + edit path if you need):
# import tensorflow as tf
# model = tf.keras.models.load_model('/path/to/your_model.h5', compile=False)
# -------------------------

# Sanity check for model
try:
    model  # reference to your keras model
except NameError:
    print("Warning: 'model' is not defined. Load your Keras model first (see comment at top of the cell).")

uploader = widgets.FileUpload(accept='image/*', multiple=False)
out = widgets.Output()
display(uploader, out)

def handle_upload(change):
    with out:
        clear_output(wait=True)
        if not uploader.value:
            print("No file uploaded yet.")
            return

        # Fetch the uploaded file robustly (handles different widget versions)
        try:
            if isinstance(uploader.value, dict):
                fname = list(uploader.value.keys())[0]
                fileinfo = uploader.value[fname]
            else:
                # some versions return a list/tuple
                fileinfo = uploader.value[0]
                fname = fileinfo.get('name', 'uploaded_image')

            print("Filename:", fname)

            # Extract bytes
            if isinstance(fileinfo, dict) and 'content' in fileinfo:
                file_bytes = fileinfo['content']
            elif isinstance(fileinfo, (bytes, bytearray)):
                file_bytes = fileinfo
            else:
                # fallback
                try:
                    file_bytes = io.BytesIO(fileinfo).getvalue()
                except Exception as e:
                    print("Could not extract bytes from upload:", e)
                    return

            # Convert bytes -> cv2 image (BGR)
            np_arr = np.frombuffer(file_bytes, np.uint8)
            img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
            if img is None:
                print("Uploaded file is not a valid image or cv2 failed to decode it.")
                return

            # Preprocess to match training (adjust if you used different preprocessing)
            img_resized = cv2.resize(img, (224, 224))
            img_input = img_resized.astype(np.float32) / 255.0
            img_input = np.expand_dims(img_input, axis=0)  # shape (1,224,224,3)

            # Run model.predict and handle errors
            try:
                pred = model.predict(img_input)
            except Exception as e:
                print("Error during model.predict():", e)
                return

            pred = np.asarray(pred)
            print("Raw model output shape:", pred.shape)

            # Decide class -> 0 is Cat, else Dog
            # Handle common output shapes:
            label_name = "Unknown"
            score = None

            if pred.ndim == 2 and pred.shape[1] > 1:
                # e.g., [[0.1, 0.9]]
                label = int(np.argmax(pred, axis=1)[0])
                score = float(np.max(pred, axis=1)[0])
                label_name = "Cat" if label == 0 else "Dog"

            elif pred.ndim == 2 and pred.shape[1] == 1:
                # e.g., [[0.8]] (sigmoid score for positive class)
                score = float(pred[0,0])
                label = int(score >= 0.5)  # 1 -> Dog, 0 -> Cat
                label_name = "Cat" if label == 0 else "Dog"

            elif pred.ndim == 1:
                # e.g., [0.1, 0.9] or [0.8]
                if pred.size > 1:
                    label = int(np.argmax(pred))
                    score = float(np.max(pred))
                    label_name = "Cat" if label == 0 else "Dog"
                else:
                    score = float(pred[0])
                    label = int(score >= 0.5)
                    label_name = "Cat" if label == 0 else "Dog"

            else:
                # Unexpected shape
                print("Unhandled model output shape; raw output printed below.")
                print(pred)
                return

            score_str = f"{score:.4f}" if score is not None else "N/A"

            # Display the image and the label
            plt.figure(figsize=(4,4))
            plt.imshow(cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB))
            plt.title(f"Prediction: {label_name} (score={score_str})")
            plt.axis('off')
            plt.show()

            print("Prediction:", label_name, f"(score={score_str})")

        except Exception as exc:
            print("Unexpected error in upload handler:", exc)

# Attach handler
uploader.observe(handle_upload, names='value')


FileUpload(value=(), accept='image/*', description='Upload')

Output()